<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Material/blob/main/Options_Pricing_with_multiple_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Import the necessary packages
import numpy as np
from scipy.stats import norm
import pandas as pd

In [ ]:
# Create a class to store the properties of an option
class OptionProperties:
  def __init__(self, S0, K, r, T, N, div=0, is_put=False, is_am=False):
    self.S0 = S0       # Initial stock price
    self.K = K         # Strike price
    self.r = r         # Risk-free rate
    self.T = T         # Time to maturity
    self.N = N         # Number of steps (for tree models)
    self.div = div     # Dividend yield
    self.is_put = is_put  # True for a put option, False for a call option
    self.is_am = is_am    # True for American, False for European

In [29]:
# Create a class to price an option using the Monte Carlo Method
class MonteCarloPricing:
  def __init__(self, option):
    self.option = option

  def price(self, M=10000):
    dt = self.option.T / self.option.N
    discount_factor = np.exp(-self.option.r * self.option.T)

    # Simulate M price paths
    prices = np.zeros(M)
    for i in range(M):
      price_path = self.option.S0
      for _ in range(self.option.N):
        z = np.random.normal()
        price_path *= np.exp((self.option.r - self.option.div - 0.5 * 0.2**2) * dt + 0.2 * np.sqrt(dt) * z)
      prices[i] = price_path

    if self.option.is_put:
      payoff = np.maximum(self.option.K - prices, 0)
    else:
      payoff = np.maximum(prices - self.option.K, 0)

    return discount_factor * np.mean(payoff)


In [30]:
# Create a class to price options using the Trinomial Tree
class TrinomialTree:
  def __init__(self, option):
    self.option = option

  def price(self):
    dt = self.option.T / self.option.N
    u = np.exp(0.2 * np.sqrt(2 * dt))  # Up factor
    d = 1 / u  # Down factor
    m = 1  # Middle factor (no change)
    q = np.exp(-self.option.div * dt)

    # Probabilities for up, down, and middle moves
    pu = ((np.exp(self.option.r * dt / 2) - np.exp(-0.2 * np.sqrt(dt / 2)))**2) / ((np.exp(0.2 * np.sqrt(dt / 2)) - np.exp(-0.2 * np.sqrt(dt / 2)))**2)
    pd = ((np.exp(0.2 * np.sqrt(dt / 2)) - np.exp(self.option.r * dt / 2))**2) / ((np.exp(0.2 * np.sqrt(dt / 2)) - np.exp(-0.2 * np.sqrt(dt / 2)))**2)
    pm = 1 - pu - pd

    # Initialize the price tree
    stock_prices = np.zeros((2 * self.option.N + 1, self.option.N + 1))
    stock_prices[self.option.N, 0] = self.option.S0

    for i in range(1, self.option.N + 1):
      for j in range(-i, i + 1, 1):
        stock_prices[self.option.N + j, i] = self.option.S0 * (u ** j)

    option_values = np.maximum(0, (self.option.K - stock_prices) if self.option.is_put else (stock_prices - self.option.K))

    for i in range(self.option.N - 1, -1, -1):
      for j in range(-i, i + 1, 1):
        option_values[self.option.N + j, i] = (pu * option_values[self.option.N + j + 1, i + 1] + pm * option_values[self.option.N + j, i + 1] + pd * option_values[self.option.N + j - 1, i + 1]) * q
        if self.option.is_am:
          option_values[self.option.N + j, i] = max(option_values[self.option.N + j, i], self.option.K - stock_prices[self.option.N + j, i] if self.option.is_put else stock_prices[self.option.N + j, i] - self.option.K)

    return option_values[self.option.N, 0]

In [ ]:
# Create a class to price options using the Cox-Ross-Rubinstein (CRR) Binomial Tree Pricing model
class CRRBinomialTree:
  def __init__(self, option):
    self.option = option

  def price(self):
    dt = self.option.T / self.option.N
    u = np.exp(0.2 * np.sqrt(dt))  # Up factor
    d = 1 / u # Down factor
    pu = (np.exp((self.option.r - self.option.div) * dt) - d) / (u - d)  # Probability of up move
    pd = 1 - pu  # Probability of down move
    discount = np.exp(-self.option.r * dt)

    # Stock price tree
    stock_prices = np.zeros((self.option.N + 1, self.option.N + 1))
    for i in range(self.option.N + 1):
      for j in range(i + 1):
        stock_prices[j, i] = self.option.S0 * (u ** (i - j)) * (d ** j)

    # Option price at maturity
    option_values = np.maximum(0, (self.option.K - stock_prices[:, self.option.N]) if self.option.is_put else (stock_prices[:, self.option.N] - self.option.K))

    # Backward iteration to get the option price
    for i in range(self.option.N - 1, -1, -1):
      for j in range(i + 1):
        option_values[j] = discount * (pu * option_values[j] + pd * option_values[j + 1])
        if self.option.is_am:
          option_values[j] = max(option_values[j], self.option.K - stock_prices[j, i] if self.option.is_put else stock_prices[j, i] - self.option.K)

    return option_values[0]

In [8]:
# A class to price options using the Finite Difference Method (Explicit Scheme) method
class FiniteDifferenceMethod:
  def __init__(self, option, M=100, Smax=2):
    self.option = option
    self.M = M
    self.Smax = Smax * option.K

  def price(self):
    dt = self.option.T / self.option.N
    ds = self.Smax / self.M
    grid = np.zeros((self.M + 1, self.option.N + 1))
    stock_prices = np.linspace(0, self.Smax, self.M + 1)

    # Boundary conditions
    if self.option.is_put:
      grid[:, -1] = np.maximum(self.option.K - stock_prices, 0)
    else:
      grid[:, -1] = np.maximum(stock_prices - self.option.K, 0)

    for i in range(self.M + 1):
      grid[i, 0] = 0 if self.option.is_put else max(stock_prices[i] - self.option.K, 0)

    alpha = 0.5 * dt * (0.2**2 * np.arange(self.M + 1)**2 - (self.option.r - self.option.div) * np.arange(self.M + 1))
    beta = -dt * (0.2**2 * np.arange(self.M + 1)**2 + self.option.r)
    gamma = 0.5 * dt * (0.2**2 * np.arange(self.M + 1)**2 + (self.option.r - self.option.div) * np.arange(self.M + 1))

    # Backward induction
    for j in range(self.option.N - 1, -1, -1):
      for i in range(1, self.M):
        grid[i, j] = alpha[i] * grid[i - 1, j + 1] + beta[i] * grid[i, j + 1] + gamma[i] * grid[i + 1, j + 1]

      # Apply early exercise condition for American options
      if self.option.is_am:
        if self.option.is_put:
          grid[:, j] = np.maximum(grid[:, j], self.option.K - stock_prices)
        else:
          grid[:, j] = np.maximum(grid[:, j], stock_prices - self.option.K)

    return np.interp(self.option.S0, stock_prices, grid[:, 0])


In [9]:
# A class to price options using the Leisen-Reimer Tree Pricing method
class LeisenReimerTree:
  def __init__(self, option):
    self.option = option

  def price(self):
    dt = self.option.T / self.option.N
    u = np.exp(0.2 * np.sqrt(dt))
    d = 1 / u
    p = 0.5  # Use Leisen-Reimer probabilities or something similar
    discount = np.exp(-self.option.r * dt)

    stock_prices = np.zeros((self.option.N + 1, self.option.N + 1))
    for i in range(self.option.N + 1):
      for j in range(i + 1):
        stock_prices[j, i] = self.option.S0 * (u ** (i - j)) * (d ** j)

    option_values = np.maximum(0, (self.option.K - stock_prices[:, self.option.N]) if self.option.is_put else (stock_prices[:, self.option.N] - self.option.K))

    for i in range(self.option.N - 1, -1, -1):
      for j in range(i + 1):
        option_values[j] = discount * (p * option_values[j] + (1 - p) * option_values[j + 1])
        if self.option.is_am:
          option_values[j] = max(option_values[j], self.option.K - stock_prices[j, i] if self.option.is_put else stock_prices[j, i] - self.option.K)

    return option_values[0]

In [11]:
# Function to price European options using the Black-Scholes-Merton (BSM) formula
def black_scholes_price(S, K, r, T, sigma, is_put=False):
  d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
  d2 = d1 - sigma * np.sqrt(T)
  if is_put:
    price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
  else:
    price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
  return price

In [12]:
# Create a function the calculate the "true" price of an option which we will use to compare with our models. We will use Black Scholes for European Options and a CRR tree with 1000 steps for American options
def true_option_price(option, sigma=0.2):
  if option.is_am:
    # Use a high number of steps in CRR for American options
    high_steps = 1000
    option_high_steps = OptionProperties(S0=option.S0, K=option.K, r=option.r, T=option.T, N=high_steps, is_put=option.is_put, is_am=True)
    crr_pricer = CRRBinomialTree(option_high_steps)
    return crr_pricer.price()
  else:
    # Use Black-Scholes for European options
    return black_scholes_price(option.S0, option.K, option.r, option.T, sigma, is_put=option.is_put)


In [24]:
# Create a function to price our options and compare them with the "true prices"
def compare_models_with_metrics():
  options = [OptionProperties(S0=100, K=100, r=0.05, T=1, N=100, is_put=False, is_am=False),
             OptionProperties(S0=100, K=100, r=0.05, T=1, N=100, is_put=True, is_am=False),
             OptionProperties(S0=100, K=100, r=0.05, T=1, N=100, is_put=False, is_am=True),
             OptionProperties(S0=100, K=100, r=0.05, T=1, N=100, is_put=True, is_am=True),
             OptionProperties(S0=100, K=105, r=0.03, T=0.5, N=50, is_put=False, is_am=False),
             OptionProperties(S0=90, K=95, r=0.04, T=2, N=200, is_put=True, is_am=True)
             ]

  # Results list to store pricing data
  results = []

  for option in options:
    # True option price (Black-Scholes for European, CRR high N for American)
    true_price = true_option_price(option)

    # Monte Carlo Pricing
    mc_pricer = MonteCarloPricing(option)
    mc_price = mc_pricer.price()

    # Trinomial Tree Pricing
    trinomial_pricer = TrinomialTree(option)
    trinomial_price = trinomial_pricer.price()

    # CRR Binomial Tree Pricing
    crr_pricer = CRRBinomialTree(option)
    crr_price = crr_pricer.price()

    # Finite Difference Method
    fdm_pricer = FiniteDifferenceMethod(option)
    fdm_price = fdm_pricer.price()

    # Leisen-Reimer Tree Pricing
    lr_pricer = LeisenReimerTree(option)
    lr_price = lr_pricer.price()

    # Collect data with metrics into the results list
    results.append({
        "Option Type": f"{'American' if option.is_am else 'European'} {'Put' if option.is_put else 'Call'}",
        "S0": option.S0,
        "K": option.K,
        "T": option.T,
        "N": option.N,
        "True Price": true_price,
        "Monte Carlo": mc_price,
        "MC Abs Error": abs(mc_price - true_price),
        "MC % Error": abs(mc_price - true_price) / true_price * 100,
        "Trinomial Tree": trinomial_price,
        "Trinomial Abs Error": abs(trinomial_price - true_price),
        "Trinomial % Error": abs(trinomial_price - true_price) / true_price * 100,
        "CRR Binomial Tree": crr_price,
        "CRR Abs Error": abs(crr_price - true_price),
        "CRR % Error": abs(crr_price - true_price) / true_price * 100,
        "Finite Difference": fdm_price,
        "FDM Abs Error": abs(fdm_price - true_price),
        "FDM % Error": abs(fdm_price - true_price) / true_price * 100,
        "Leisen-Reimer Tree": lr_price,
        "LR Abs Error": abs(lr_price - true_price),
        "LR % Error": abs(lr_price - true_price) / true_price * 100
        })
  # Convert the results list into a DataFrame for easy viewing
  df = pd.DataFrame(results)
  return df

In [26]:
# Use the compare metrics for all types of pricing methods for each of the options in the compare with metrics functions

df = compare_models_with_metrics()
print(df)


     Option Type   S0    K    T    N  True Price  Monte Carlo  MC Abs Error  \
0  European Call  100  100  1.0  100   10.450584    10.555242      0.104658   
1   European Put  100  100  1.0  100    5.573526     5.557479      0.016047   
2  American Call  100  100  1.0  100   10.448584    10.476777      0.028193   
3   American Put  100  100  1.0  100    6.089595     5.656326      0.433270   
4  European Call  100  105  0.5   50    4.178300     4.260808      0.082508   
5   American Put   90   95  2.0  200   10.059935     8.950530      1.109405   

   MC % Error  Trinomial Tree  ...  Trinomial % Error  CRR Binomial Tree  \
0    1.001456       10.975892  ...           5.026592          10.430612   
1    0.287913        5.848782  ...           4.938636           5.553554   
2    0.269823       10.975892  ...           5.046691          10.430612   
3    7.114919        6.256466  ...           2.740260           6.082354   
4    1.974686        4.248928  ...           1.690355           4.